In [1]:
import numpy as np; np.random.seed(0)
import seaborn as sns;
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import os
import scipy.stats
import math
from ete3 import Tree

%matplotlib inline  
pylab.rcParams['figure.figsize'] = 15, 15  # that's default image size for this

import json

In [2]:
# for each line in the embryo file, split out the HMID into individual events and add their counts together
embryo_data = open("/mount/vol10/CRISPR.lineage/nobackup/2016_05_04_embryo_rerun/merged_v7_embryo_data_gt_100_May_5th_2016.txt")
header = embryo_data.readline()
embryo_to_event_counts = {}
embryo_to_counts = {}
embryo_to_unique_counts = {}

counted_tokens = 0
for line in embryo_data:
    sp = line.strip("\n").split("\t")
    sample = sp[0]
    hmid = sp[4]
    count = int(sp[6])
    event_list = set()
    
    for token in hmid.split("_"):
        if "&" in token:
            for tk in token.split("&"):
                if tk != "NONE":
                    event_list.add(tk)
        else:
            event_list.add(token)
                
    for event in event_list:
        if not sample in embryo_to_event_counts:
            embryo_to_event_counts[sample] = {}
        counted_tokens += count
        embryo_to_event_counts[sample][event] = embryo_to_event_counts[sample].get(event,0) + count
        
    embryo_to_counts[sample] = embryo_to_counts.get(sample,0) + count
    embryo_to_unique_counts[sample] = embryo_to_unique_counts.get(sample,0) + 1


In [3]:
embryo_to_event_counts['Dome_1_1x']['20I+140+TACTGTTTTAGATACTGTTT']
# embryo_to_event_counts['Dome_1_1x']


744

In [4]:
# filter down the sample list
embryo_to_use = []
for embryo,count in embryo_to_counts.iteritems():
    if not "1b" in embryo and count >= 150: # and embryo_to_unique_counts[embryo] >= 20:
        embryo_to_use.append(embryo)

In [5]:
len(set(embryo_to_use))

45

In [6]:
# for each embryo, find the most prominant edit.  Then find their proportion each other embryo, record each
top_edits = open("top_events_per_embryo_new.txt","w")
top_edits.write("sample\tevent\tpropotions\ttype\n")

top_event_per_embryo = {}
top_event_per_embryo_count = {}

for sample in embryo_to_use:
    top_event_per_embryo[sample] = "NOPE"
    top_event_per_embryo_count[sample] = 0
    
    for event, count in embryo_to_event_counts[sample].iteritems():
        # print count
        if count > top_event_per_embryo_count[sample] and event != "NONE":
            top_event_per_embryo[sample] = event
            top_event_per_embryo_count[sample] = count
            

top_event_to_samples = {}
for sample in embryo_to_use:
    top_event = top_event_per_embryo[sample]
    
    if not top_event in top_event_to_samples:
        top_event_to_samples[top_event] = []
    top_event_to_samples[top_event].append(sample)
    
    prop = float(top_event_per_embryo_count[sample]) / float(embryo_to_counts[sample])
    top_edits.write(sample + "\t" + top_event_per_embryo[sample] + "\t" + str(prop) + "\ttop\n")


In [7]:
event_to_rate_in_other = {}
event_to_count_in_other = {}
for sample in embryo_to_use:
    for event, count in embryo_to_event_counts[sample].iteritems():
        if event in top_event_to_samples and not sample in top_event_to_samples[event]:
            prop = float(embryo_to_event_counts[sample][event]) / float(embryo_to_counts[sample])
            event_to_rate_in_other[event] = event_to_rate_in_other.get(event,0.0) + prop 
            event_to_count_in_other[event] = event_to_count_in_other.get(event,0) + 1

for sample, event in top_event_per_embryo.iteritems():
    rate = event_to_rate_in_other.get(event,0.0)
    count = event_to_count_in_other.get(event,1)
    avg_rate_in_other = float(rate) / float(count)
    top_edits.write(sample + "\t" + event + "\t" + str(avg_rate_in_other) + "\tother\n")

In [8]:

top_edits.close()